# Coef Analysis of linear Regression

In [12]:
import os
import pickle

# imports
import pandas as pd
from sklearn.model_selection import train_test_split

## Getting Data

In [14]:
data = pd.read_csv('new_Exam_Score_Prediction.csv')

y = data['exam_score']
X = data.drop('exam_score', axis=1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

x_train

,age,gender,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,facility_rating,exam_difficulty,diploma,group_study,self_study,online_videos,coaching,mixed
3207,20,1,6.48,82.6,0,8.4,0,0,2,0,0,0,0,0,1
1590,18,0,0.68,95.5,1,4.7,2,2,1,0,0,0,1,0,0
593,20,0,0.97,81.5,0,7.9,2,0,1,0,0,0,1,0,0
4707,20,1,7.70,70.4,1,9.7,2,0,2,0,1,0,0,0,0
8571,20,0,1.98,60.4,0,5.8,2,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,24,1,2.24,42.4,1,4.4,1,1,2,0,0,0,0,0,1
3264,24,0,0.86,95.8,1,5.8,0,0,0,0,0,0,1,0,0
9845,22,0,6.32,95.6,1,9.5,0,1,0,0,0,0,0,1,0
10799,22,1,7.71,84.1,1,7.9,2,1,2,0,0,1,0,0,0


# Getting Model

In [18]:
with open(os.path.join('pipes', 'Linear.pkl'), 'rb') as f:
    Pipe = pickle.load(f)

Pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('features_selector',
                 SelectKBest(score_func=<function f_regression at 0x76a2b9975a80>)),
                ('regressor', LinearRegression())])

# T-test for Regression Coefficients

In [47]:
import numpy as np
import statsmodels.api as sm

names = x_train.columns
selector = Pipe.named_steps['features_selector']
selected_names = np.array(names)[mask]

test_x = x_train[selected_names]
test_y = y_train


X_constant = sm.add_constant(test_x)
sm_model = sm.OLS(test_y, X_constant).fit()
print(sm_model.summary())

                            OLS Regression Results                            
Dep. Variable:             exam_score   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     4004.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        01:21:30   Log-Likelihood:                -49114.
No. Observations:               13274   AIC:                         9.825e+04
Df Residuals:                   13264   BIC:                         9.832e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.8918      0.445  

# Drop Mixed studing method

In [57]:
names = x_train.columns
selector = Pipe.named_steps['features_selector']
selected_names = np.array(names)[mask]
selected_names = selected_names[selected_names != 'mixed']

test_x = x_train[selected_names]
test_y = y_train


X_constant = sm.add_constant(test_x)
sm_model2 = sm.OLS(test_y, X_constant).fit()
print(sm_model2.summary())

                            OLS Regression Results                            
Dep. Variable:             exam_score   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     2849.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        01:28:23   Log-Likelihood:                -34310.
No. Observations:                9291   AIC:                         6.864e+04
Df Residuals:                    9281   BIC:                         6.871e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.8466      0.669  

# Test data

## With mixed

In [58]:
names = x_train.columns
selector = Pipe.named_steps['features_selector']
selected_names = np.array(names)[mask]

test_x = x_test[selected_names]
test_y = y_test


X_constant = sm.add_constant(test_x)
sm_model = sm.OLS(test_y, X_constant).fit()
print(sm_model.summary())

                            OLS Regression Results                            
Dep. Variable:             exam_score   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     1159.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        01:29:24   Log-Likelihood:                -14796.
No. Observations:                3983   AIC:                         2.961e+04
Df Residuals:                    3973   BIC:                         2.967e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.0417      0.816  

## Without Mixed

In [60]:
names = x_train.columns
selector = Pipe.named_steps['features_selector']
selected_names = np.array(names)[mask]
selected_names = selected_names[selected_names != 'mixed']

test_x = x_test[selected_names]
test_y = y_test


X_constant = sm.add_constant(test_x)
sm_model2 = sm.OLS(test_y, X_constant).fit()
print(sm_model2.summary())

                            OLS Regression Results                            
Dep. Variable:             exam_score   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                     1159.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        01:29:42   Log-Likelihood:                -14796.
No. Observations:                3983   AIC:                         2.961e+04
Df Residuals:                    3973   BIC:                         2.967e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.0432      1.041  